In [ ]:
from utils.binaries import *
from utils.plotting import *
from utils.Auger.FD import PixelPlot, XYScanner

In [ ]:
import os
from collections import defaultdict

SCANNER_RUNLISTS = "/cr/data01/filip/xy-calibration/config/calib_runlists/"


def reject(run):
    if not run: return True
    # elif run == '16407': return False      # test
    
    else:
        return False

run_data = defaultdict(list)
for LIST in os.listdir(SCANNER_RUNLISTS):
    if 'va' in LIST or 'testing' in LIST: continue

    runlist = XYScanner.load_runlist(LIST[11:18])
    runlist = XYScanner.get_good_runs(runlist)

    for run, info in runlist.iterrows():
        if reject(run): continue

        run_data[info['tel'].upper()].append([run, info['date']])

In [ ]:
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

for tel, runs in run_data.items():
    if not "CO" in tel: continue

    n_measurements = len(runs)

    fig, axes = plt.subplots(1, n_measurements + 1, figsize=[n_measurements * 3.3, 3.3], width_ratios=[1] * n_measurements + [0.05])
    for ax, (_id, _date) in zip(axes, runs): 
        ax.set_title(_date)

        std, xy = np.loadtxt(f'/cr/data01/filip/xy-calibration/results/outCorr_{_id}.txt', usecols=[3, 4], unpack=True)
        PixelPlot(ratio := xy / std, ax=ax, cmap='coolwarm', vmin=0, vmax=2)

        ax.text(0.03, 0.97, fr"$\tilde{{r}}$ = {np.median(ratio):.3f}",
                 ha='left',
                 va='top',
                 fontsize='x-large',
                 transform=ax.transAxes,
                 bbox=dict(boxstyle="round",ec='none',fc='white',alpha=0.7))

    plt.subplots_adjust(wspace=0.04)
    axes[0].text(-0.07, 0.5, tel,
                 ha='center',
                 va='center',
                 rotation=90,
                 transform=axes[0].transAxes,
                 fontsize='xx-large')
    plt.subplots_adjust(wspace=0.04)
    plt.colorbar(ScalarMappable(cmap='coolwarm', norm=Normalize(0, 2)), cax=axes[-1], shrink=0.7, label="XY calib. corr. / std. calib.")


    # break
